In [1]:
import hnswlib
import numpy as np

In [2]:
dim = 5
num_elements = 10
num_nei = 5
# Generating sample data
data = np.float32(np.random.random((num_elements, dim)))
data_labels = np.arange(num_elements)

# Declaring index
p = hnswlib.Index(space = 'l2', dim = dim) # possible options are l2, cosine or ip

# Initing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 200, M = 16)

# Element insertion (can be called several times):
p.add_items(data, data_labels)

# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

# Query dataset, k - number of closest elements (returns 2 numpy arrays)
labels, distances = p.knn_query(data, k = num_nei)

In [15]:
print(labels)
print(distances)

[[0 8 4 2 3]
 [1 2 4 8 0]
 [2 8 4 1 9]
 [3 0 4 2 8]
 [4 2 8 0 9]
 [5 9 2 8 1]
 [6 7 4 1 2]
 [7 4 6 9 8]
 [8 2 4 0 1]
 [9 5 2 4 8]]
[[0.         0.5565159  0.5981668  0.6174321  0.67154723]
 [0.         0.51579463 0.6668134  0.71150196 0.8091073 ]
 [0.         0.41812053 0.45895582 0.51579463 0.54268676]
 [0.         0.67154723 0.7538125  1.0567069  1.0663426 ]
 [0.         0.45895582 0.5292437  0.5981668  0.6465419 ]
 [0.         0.54015934 0.580719   0.72352177 0.8274091 ]
 [0.         0.76762354 0.79124206 0.83210987 1.0172647 ]
 [0.         0.67817444 0.76762354 0.98682976 1.0448397 ]
 [0.         0.41812053 0.5292437  0.5565159  0.71150196]
 [0.         0.54015934 0.54268676 0.6465419  0.71367615]]


In [4]:
from sklearn.utils.graph import graph_shortest_path
from sklearn.neighbors import NearestNeighbors, kneighbors_graph

In [5]:
def hard_geodesics_euclidean_kernel_regular(features, n_neighbors):
    # Regular
    nbrs_ = NearestNeighbors(n_neighbors=n_neighbors,
                             algorithm='auto',
                             metric='euclidean',
                             n_jobs=2)
    nbrs_.fit(features)
    kng = kneighbors_graph(X=nbrs_, n_neighbors=n_neighbors,
                           metric='euclidean',
                           mode='distance', n_jobs=2)
    dist_matrix_ = graph_shortest_path(kng,
                                       method='FW',
                                       directed=False)
    kernel = (0.5)*dist_matrix_**2
    max_distance = np.max(kernel)+1
    kernel[kernel == 0]=max_distance
    return kernel, kng

In [11]:
a,b=hard_geodesics_euclidean_kernel_regular(data, 3)

In [ ]:
np.linalg.norm(data[5]-data[0])

In [12]:
print(b)

  (0, 8)	0.5565158927928648
  (0, 4)	0.5981667952274659
  (0, 2)	0.617432115563808
  (1, 2)	0.5157946311747941
  (1, 4)	0.6668133456479236
  (1, 8)	0.711501962731765
  (2, 8)	0.41812052378390047
  (2, 4)	0.45895582479734404
  (2, 1)	0.5157946311747941
  (3, 0)	0.6715472926084782
  (3, 4)	0.7538125507225271
  (3, 2)	1.0567068489089755
  (4, 2)	0.45895582479734404
  (4, 8)	0.5292436871272816
  (4, 0)	0.5981667952274659
  (5, 9)	0.5401593008668456
  (5, 2)	0.5807190201027894
  (5, 8)	0.7235217737714046
  (6, 7)	0.7676235516148121
  (6, 4)	0.791242093004135
  (6, 1)	0.8321098755050114
  (7, 4)	0.6781744706028083
  (7, 6)	0.7676235516148121
  (7, 9)	0.986829739664638
  (8, 2)	0.41812052378390047
  (8, 4)	0.5292436871272816
  (8, 0)	0.5565158927928648
  (9, 5)	0.5401593008668456
  (9, 2)	0.5426867531568511
  (9, 4)	0.6465418870658352


In [9]:
np.linalg.norm(data[8]-data[0])*np.linalg.norm(data[8]-data[0])

0.30971

In [10]:
np.linalg.norm(data[8]-data[0])

0.55651593